# P 9.1

Write the function `PollardRho(g, h, p)` in SAGE that solves the discrete logarithm problem $g^x \equiv h \mod p$ in $\mathbb{F}_p^{\times}$ for a generator $g$ of $\mathbb{F}_p^{\times}$, using the base point $x_0 = 1$ and
the partition $\mathbb{F}_p^{\times} = P_0 \sqcup P_1 \sqcup P_2$ where $P_j := \{ x \ (\text{mod } p) : jp/3 \leq x < (j + 1)p/3 \}$ for $j \in \{0,1,2 \}$. 

You may want to use exercise T9.2.

In [171]:
def determine_partition(x,p):
    if 0 <= float(x) and float(x) < p/3:
        return 0   # x in P_0
    if p/3 <= float(x) and float(x) < 2*p/3:
        return 1    # x in P_1
    if 2*p/3 <= float(x) and float(x) < p:
        return 2    # x in P_2

def evaluate_f(x, g, h, p):
    partition = determine_partition(x,p)
    if partition == 0:
        return g*x % p
    if partition == 1:
        return x**2 % p
    if partition == 2:
        return h*x % p

def update_a(a, x, p):
    partition = determine_partition(x,p)
    if partition == 0:
        return (a + 1) % (p - 1)
    if partition == 1:
        return 2*a % (p - 1)
    if partition == 2:
        return a

def update_b(b, x, p):
    partition = determine_partition(x,p)
    if partition == 0:
        return b
    if partition == 1:
        return 2*b % (p - 1)
    if partition == 2:
        return (b + 1) % (p - 1)

In [172]:
class Tortoise:
    def __init__(self, x, a, b):
        self.x = x
        self.a = a
        self.b = b

class Hare:
    def __init__(self, x, a, b):
        self.x = x
        self.a = a
        self.b = b

class TempValue:
    def __init__(self, x, a, b):
        self.x = x
        self.a = a
        self.b = b

BOUND = 2048

def PollardRho(g, h, p):
    tortoise = Tortoise(1, 0, 0)
    hare = Hare(1, 0, 0)
    temp = TempValue(1, 0, 0)

    for _ in range(1, BOUND):
        tortoise.a = update_a(tortoise.a, tortoise.x, p)
        tortoise.b = update_b(tortoise.b, tortoise.x, p)
        tortoise.x = evaluate_f(tortoise.x, g, h, p)
        tortoise = Tortoise(tortoise.x, tortoise.a, tortoise.b)

        temp.x = evaluate_f(hare.x, g, h, p)
        temp.a = update_a(hare.a, hare.x, p)
        temp.b = update_b(hare.b, hare.x, p)
        temp = TempValue(temp.x, temp.a, temp.b)

        hare.x = evaluate_f(temp.x, g, h, p)
        hare.a = update_a(temp.a, temp.x, p)
        hare.b = update_b(temp.b, temp.x, p)
        hare = Hare(hare.x, hare.a, hare.b)

        if tortoise.x == hare.x:
            if (tortoise.b - hare.b) == 0:
                raise ValueError("Error: Division by zero")
            results = solve_mod((tortoise.b - hare.b)*x == (hare.a - tortoise.a), p - 1)
            for element in results:
                if (g**element[0]) % p == h:
                    return element[0]
            raise ValueError("Error: No solution found")
    
    raise ValueError("Error: No solution found")

In [177]:
import time

# Generate a random finite field F_p
print("Let F_p = <g> be a random finite field and h a random element of F_p:")
p = 1
while not is_prime(p):
    p = randint(2**12, 2**20)
F = GF(p)

# Pick a random generator of F_p
g = F.multiplicative_generator()

# Pick a random h
h = 0
while h == 0:
    h = F.random_element()

print(f"\tp = {p}\n\tg = {g}\n\th = {h}")

# Solve the discrete logarithm
start = time.time() # start a timer
result = PollardRho(g, h, p)
print(f"\nSolution of the discrete logarithm:\n\tlog_{g}({h}) = {result}.")
end = time.time() # stop the timer

if (g**result) % p == h:
    print(f"\nThe solution is correct and was computed in {round(end - start,3)} seconds!")

Let F_p = <g> be a random finite field and h a random element of F_p:
	p = 170389
	g = 2
	h = 8284

Solution of the discrete logarithm:
	log_2(8284) = 75308.

The solution is correct and was computed in 9.804 seconds!
